In [ ]:
!pip install -r ./requirements.txt

In [ ]:
import numpy as np
import scipy as sc
import scipy.sparse as sp
import random
from collections import defaultdict
from pyspark.sql import SparkSession

spark = (SparkSession 
         .builder
         .appName("Cloud computing ENSAE project")
         .master("local[5]")
         .getOrCreate()
        )
sc = spark.sparkContext

In [ ]:
def mult(x):
    y = np.array([2])
    return x*y
 
x = np.arange(10000)
distData = sc.parallelize(x)
 
results = distData.map(mult).collect()
 

In [ ]:
spark.stop()

In [ ]:
def generate_sparse_matrix_dict_repr(m,n,num_nonzero):
    dict_repr=dict()
    
    i,j = np.random.choice(m, num_nonzero, replace=True) , np.random.choice(n, num_nonzero, replace=True)
    for x,y in zip(i,j):
        dict_repr[(x,y)]=1
    return dict_repr

In [ ]:
def dot_product_with_dict_repr(dict_repr,m,n):
    multiplied_dict_repr=dict()
    for i in range(n):
        dict_col_i={x:v for (x,y),v in dict_repr.items() if y==i}
        dict_col_i=defaultdict(int,dict_col_i)
        if dict_col_i:
            for j in range(m):
                result=0
                dict_col_j={x:v for (x,y),v in dict_repr.items() if y==j and x in dict_col_i.keys()}
                dict_col_j=defaultdict(int,dict_col_j)
                if dict_col_j:
                    for x,v in dict_col_i.items():
                        result+= v*dict_col_j[x]
                    if result!=0:
                        multiplied_dict_repr[(i,j)]=result
    return multiplied_dict_repr

In [ ]:
a=generate_sparse_matrix_dict_repr(300,40,520)

In [ ]:
def dict_repr_to_sp_csc(dict_rep,m,n):
    row = np.array([])
    col = np.array([])
    data = np.array([])

    for (x,y),v in dict_rep.items():
        row=np.append(row,x)
        col=np.append(col,y)
        data=np.append(data,v)
    return sp.csc_matrix((data, (row, col)), shape=(m, n))

In [ ]:
dict_repr_to_sp_csc(dot_product_with_dict_repr(a,300,40),40,40).toarray()

In [ ]:
sparse_matrix=dict_repr_to_sp_csc(a,300,40)

In [ ]:
sparse_matrix.T@sparse_matrix.toarray()